In [1]:
"""
  Converting the dataset into the required format
  #https://www.kaggle.com/datasets/bfbarry/haiku-dataset
"""

'\n  Converting the dataset into the required format\n  #https://www.kaggle.com/datasets/bfbarry/haiku-dataset\n'

In [ ]:
!pip install --upgrade fastpunct

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 36.8 MB/s 
     |████████████████████████████████| 5.5 MB 56.0 MB/s 
     |████████████████████████████████| 182 kB 73.1 MB/s 
     |██████████████████▎             | 4.3 MB 56.2 MB/s eta 0:00:01

In [ ]:
!pip install  --upgrade  flair

In [ ]:
! pip install  --upgrade  keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.0 MB/s 
     |████████████████████████████████| 237 kB 62.9 MB/s 
     |████████████████████████████████| 51 kB 7.6 MB/s 
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23799 sha256=5c4ea868ebbbf0c93cbe81889629890f0c933d8626ef39a8320b24619c3db49e
  Stored in directory: /root/.cache/pip/wheels/85/0d/12/77d219f3ebbb22dc22234b4d665886c0eace86a26eca0aa72b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=286218c7bb3c1cd1131e1b1dc607c5589bb9d316e25a294d7de08bb3a719e64a
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built keybert sentence-transformers


In [ ]:
!sudo apt-get install festival espeak-ng mbrola &> /dev/null

In [ ]:
!pip install   --upgrade  phonemizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached importlib_metadata-5.0.0-py3-none-any.whl (21 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.1
    Uninstalling importlib-metadata-3.10.1:
      Successfully uninstalled importlib-metadata-3.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
konoha 4.6.5 requires importlib-metadata<4.0.0,>=3.7.0, but you have importlib-metadata 5.0.0 which is incompatible.


In [ ]:
import pandas as pd
from fastpunct import FastPunct
from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings
from phonemizer import phonemize
from phonemizer.separator import Separator

In [ ]:
!gdown --id 16lrcFNTRqMK-SClKANLFD25q1aV7ainb

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16lrcFNTRqMK-SClKANLFD25q1aV7ainb
To: /content/haiku_1.txt
100% 833k/833k [00:00<00:00, 89.7MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/CIS530-Project/Data/haikus.csv")
fastpunct = FastPunct()
#kw_model = KeyBERT()
roberta = TransformerDocumentEmbeddings('roberta-base')
kw_model = KeyBERT(model=roberta)

# print(len(data))
#0-47712 , 47712 -95424, 95424- 143136
#data = data[0:47712]

start = 0
count_split=0
while start < len(data):
    final_dataframe = pd.DataFrame()
    end = start + 3389
    split_data = data[start:end]
    count_t = 0
    for index, row in split_data.iterrows():
        print(count_split , ":::", count_t)
        count_t +=1
        haiku = [row["0"], row["1"], row["2"]]
        source = row["source"]
        
        # Correcting punctuation
        if len(np.where(np.array(haiku).astype(str)==str(np.nan))[0]) > 0:
            print("Continuing as found a NaN", haiku)
            continue
        haiku = fastpunct.punct(haiku) #TODO ,correct=True)  this is changing a few words

        # Finding topics
        keywords = kw_model.extract_keywords(" ".join(haiku), keyphrase_ngram_range=(1, 2), stop_words=None)

        if len(keywords) == 0 :
            keywords = [('the', 1)]
            
        counts = []
        line_phn = []
        for l in haiku:
            count = 0
            phn = phonemize(l, language='en-us', backend='festival',
                            with_stress=False, separator=Separator(phone=None,
                            word=' ', syllable="|"), strip=True)
            words = phn.split(" ")
            line_phn.append(phn)
            for w in words:
              syllables = w.split("|")
              count += len(syllables)
            counts.append(count)

        final_dataframe = final_dataframe.append(pd.Series(haiku + [source] + [keywords[0][0]] + counts + line_phn), ignore_index=True)
    final_dataframe.to_csv("/content/drive/MyDrive/CIS530-Project/Data/Cleanup/csv_data/cleaned-data-csv-test" + str(count_split) +".csv", index=False)
    start = end
    count_split +=1
    


['Memorial Day --', 'a shadow for each', 'white cross']
['spring rain -', 'as the doctor speaks', 'i think of lilacs']
['spring moonset --', 'a rice ball for', 'breakfast']


## BIBTEXT

In [ ]:
@misc{grootendorst2020keybert,
  author       = {Maarten Grootendorst},
  title        = {KeyBERT: Minimal keyword extraction with BERT.},
  year         = 2020,
  publisher    = {Zenodo},
  version      = {v0.3.0},
  doi          = {10.5281/zenodo.4461265},
  url          = {https://doi.org/10.5281/zenodo.4461265}
}